In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import geopandas as gpd
import math
from dask_gateway import Gateway

gateway = Gateway()
options = gateway.cluster_options()
options

Options<image='registry.eofactory.ai:5000/dhub/pangeo-vscode:latest',
        worker_cores=2,
        worker_memory=4.0,
        gpu=False>


In [25]:
import numpy as np
import xarray as xr
import stackstac
import rioxarray
import os
from dask.distributed import Client, progress
list_clusters = gateway.list_clusters()
if list_clusters:
    cluster = gateway.connect(list_clusters[0].name)
else:
    cluster = gateway.new_cluster(options)
cluster

In [18]:
cluster.shutdown()

In [26]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: /services/dask-gateway/clusters/dhub.cd93c095653048318c394335379b92f1/status,


In [5]:
import pystac_client
api = pystac_client.Client.open("https://stactest.eofactory.ai")

search_2023 = api.search(
    datetime=["2022-12-25","2023-11"],
    collections=["sentinel-1-rtc-mosaic-andhra-pradesh"],
    max_items=27
)
items = search_2023.item_collection()
items
    

In [23]:
def predict_chunk(chunk):
    import pandas as pd
    import dask.array as da 
    tmp=10*np.log10(chunk) #
    tmp=tmp.to_numpy()
    times, height, width = chunk.shape
    print(times, height, width)
    tmp1=tmp.reshape(times, height*width).T
    print(tmp1.shape)
    tmp1[np.isnan(tmp1)] = 0
    tmp1[np.finfo(np.float32).max < tmp1] = 1
    tmp1[np.finfo(np.float32).min > tmp1] = 0
    # print(tmp1.shape)
    df = pd.DataFrame(data=tmp1)
    df = df.interpolate(method='linear', limit_direction='both', axis=0)
    tmp1 = df.to_numpy()
    print(tmp1.shape)
    result = model.predict(kernel.transform(tmp1))
    chunk_result = chunk.isel(time=0)
    chunk_result.data = result.T.reshape(height, width)
    return chunk_result.astype("uint8")
    
def predict(items, bbox_4326, crop_land):
    import stackstac
    # signed_items = [pc.sign(item).to_dict() for item in items]
    # dict_items = [item.to_dict() for item in items]
    data = stackstac.stack(
            items,
            bounds_latlon=bbox_4326,
            dtype='float32',
            assets=["VH"],  # vh only
            chunksize=(27,1,256,256),
            resolution=10,
        ).where(lambda x: x > -32768, other=np.nan)  # sentinel-1-rtc uses -32768 as nodata
        # .assign_coords(band=lambda x: x.common_name.rename("band"))  # use common names
    data = data.rio.clip(crop_land.geometry)
    cliped = data.squeeze(axis=1)
    array = cliped.chunk({'x': 256,'y':256 , 'time':27}) 
    predictor = array.map_blocks(predict_chunk, template= array.isel(time=0).astype("uint8"))
    
    return predictor.compute()
import pickle
model_filekernel = '../training/kernel_poly.sav'
model_filename = '../training/linearsvc_model_poly.sav'

# load the model from disk
kernel= pickle.load(open(model_filekernel, 'rb'))
model = pickle.load(open(model_filename, 'rb'))

model, kernel

(Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('linearsvc',
                  LinearSVC(max_iter=10000, random_state=0, tol=1e-05))]),
 PolynomialCountSketch(degree=3, gamma=1.2, n_components=270, random_state=1))

In [7]:
def genarate_grid(bbox, size):
    x = math.ceil((bbox[2] - bbox[0]) / size)
    y = math.ceil((bbox[3] - bbox[1]) / size)
    x, y 
    box_grids = []
    for i in range(x):
        for j in range(y):
            box_grids.append([bbox[0]+ size*i, bbox[1]+ size*j, bbox[0]+ size*(i+1), bbox[1]+ size*(j+1)])
    return box_grids

In [8]:
aoi = gpd.read_file("../aoi/AndhraPradesh.geojson")
aoi = aoi.to_crs(epsg=32644).copy()
aoi = aoi.assign(geometry=aoi.buffer(1000))
aoi.total_bounds

array([  42451.78651023, 1395875.45449989,  897430.20059997,
       2124249.02698147])

In [11]:
box_grids[12]

[42451.786510231716, 1875875.4544998915, 82451.78651023172, 1915875.4544998915]

In [10]:
box_grids  = genarate_grid(aoi.total_bounds,40000)
print(len(box_grids))

418


In [27]:
from osgeo import gdal
predict_mask = 1
run = True
idx = 0
import rasterio
from rasterio.warp import transform_bounds
for grid in box_grids:
    idx = idx+ 1
    src = gdal.Open(f"../masks/{idx}.tif")
    if src:
        continue
    if (idx > -1):
        print(idx)
        predict_land = gpd.read_file("../farm/smoothed_andhra_pradesh.shp",bbox=grid)
        if predict_land.shape[0] > 0:
            done = False
            while not done:
                try:
                    bbox = predict_land.total_bounds
                    bbox_4326 = np.array(transform_bounds(32644, 4326, *grid))
                    predict_mask = predict(items, bbox_4326, predict_land)
                    predict_mask.rio.to_raster(f"../masks/{idx}.tif", driver="COG", tiled="YES", windows=True)
                    done = True
                except Exception:
                    print('Predict failed! Run again')
            # run = False
        
# predict_mask

1
2
3
11
12
13


ERROR 4: ../masks/1.tif: No such file or directory
ERROR 4: ../masks/2.tif: No such file or directory
ERROR 4: ../masks/3.tif: No such file or directory
ERROR 4: ../masks/11.tif: No such file or directory
ERROR 4: ../masks/12.tif: No such file or directory
ERROR 4: ../masks/13.tif: No such file or directory


14
15
16
17
18


ERROR 4: ../masks/14.tif: No such file or directory
ERROR 4: ../masks/15.tif: No such file or directory
ERROR 4: ../masks/16.tif: No such file or directory
ERROR 4: ../masks/17.tif: No such file or directory
ERROR 4: ../masks/18.tif: No such file or directory


19
20
21
30
31
32


...

ERROR 4: ../masks/402.tif: No such file or directory
ERROR 4: ../masks/403.tif: No such file or directory
ERROR 4: ../masks/404.tif: No such file or directory
ERROR 4: ../masks/405.tif: No such file or directory
ERROR 4: ../masks/406.tif: No such file or directory
ERROR 4: ../masks/407.tif: No such file or directory


407
408
409
410
411
412


ERROR 4: ../masks/408.tif: No such file or directory
ERROR 4: ../masks/409.tif: No such file or directory
ERROR 4: ../masks/410.tif: No such file or directory
ERROR 4: ../masks/411.tif: No such file or directory
ERROR 4: ../masks/412.tif: No such file or directory
ERROR 4: ../masks/413.tif: No such file or directory


413
414
415
416


ERROR 4: ../masks/414.tif: No such file or directory
ERROR 4: ../masks/415.tif: No such file or directory
ERROR 4: ../masks/416.tif: No such file or directory
